In [1]:
import freud
import gsd
import gsd.pygsd
import gsd.hoomd
import signac
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cycler

In [2]:
def atom_type_pos(snap, atom_type):
    if not isinstance(atom_type, list):
        atom_type = [atom_type]
    positions = []
    for atom in atom_type:
        indices = np.where(snap.particles.typeid == snap.particles.types.index(atom))
        positions.append(snap.particles.position[indices])
    return np.concatenate(positions)

def msd_from_gsd(gsdfile, start=-30, stop=-1, atom_type='ss', msd_mode = "window"):
    f = gsd.pygsd.GSDFile(open(gsdfile, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    positions = []
    for frame in trajectory[start:stop]:
        if atom_type == 'all':
            atom_positions = frame.particles.position[:]
        else:
            atom_positions = atom_type_pos(frame, atom_type)
        positions.append(atom_positions)
    msd = freud.msd.MSD(box=trajectory[-1].configuration.box, mode=msd_mode)
    msd.compute(positions)
    f.close()
    return(msd.msd) # This is retruning an array of numbers (the y-axis on the msd plot)

In [45]:
project = signac.get_project("ptb7-project")
state_dict = {"size": "small",
             "density": 0.7,
             "molecule": "PTB7_5mer_smiles",
             "process": "quench"}

In [46]:
%%time
state_dict['kT_reduced.$lt'] = 2.5

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.2 µs


In [47]:
job_list = project.find_jobs(state_dict)
fig = plt.figure()
ax = plt.subplot(111)
color = plt.cm.coolwarm(np.linspace(0, 1, len(job_list)))
mpl.rcParams['axes.prop_cycle'] = cycler.cycler('color', color)
for kT,job in job_list.groupby('kT_reduced'):
    job = list(job)[0]
    msd = msd_from_gsd(job.fn('trajectory.gsd'), atom_type='ss')
    plt.plot(msd, label=kT)

plt.title("MSD of PTB7 temperatures at %sden" % (job.sp['density']))
plt.xlabel("timestep", fontsize=14)
plt.ylabel("msd", fontsize=14)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
plt.show()


ZeroDivisionError: integer division or modulo by zero

<Figure size 432x288 with 0 Axes>

In [6]:
'''ALL OBSERVATIONS

At 0.6den:
Lower temp = lower msd
1.3kT has significantly different(higher) MSD at 0.6den

At 0.7den:
Lower temp = lower msd
1.3kT and 1.2kT msds are very similar at 0.7den 

At 0.8den:
1.4kT MSD shows up at 0.8den, weird.
These plots are kind of all over the place. 1.2kT with highest slope/msd. 0.9kT and 1.1kT with similar msds.

At 0.9den:
Lower temp = lower msd(kind of)
The grouped temps have lower temp on tomp of higher temp
1.4kT and 1.3kT identical, 1.0kT and 1.1kT identical, 0.8kT and 0.9kT identical, and 1.2kT solo.

At 1.0den:
Lower temp = lower msd(kind of)
1.2kT, 1.3kT, 1.4kT clumped together but 1.3kT is on top
0.9kT and 1.0kT indentical both taking turns but 0.9kT is on top

At 1.1den:
Lower temp = lower msd(with one exception)
1.3kT higher msd than 1.4kT

At 1.2den:
Lower temp = lower msd(until after 1.1kT)
Then order of least to greatest goes: 1.4kT, 1.2kT, 1.3kT(top)
'''

'ALL OBSERVATIONS\n\nAt 0.6den:\nLower temp = lower msd\n1.3kT has significantly different(higher) MSD at 0.6den\n\nAt 0.7den:\nLower temp = lower msd\n1.3kT and 1.2kT msds are very similar at 0.7den \n\nAt 0.8den:\n1.4kT MSD shows up at 0.8den, weird.\nThese plots are kind of all over the place. 1.2kT with highest slope/msd. 0.9kT and 1.1kT with similar msds.\n\nAt 0.9den:\nLower temp = lower msd(kind of)\nThe grouped temps have lower temp on tomp of higher temp\n1.4kT and 1.3kT identical, 1.0kT and 1.1kT identical, 0.8kT and 0.9kT identical, and 1.2kT solo.\n\nAt 1.0den:\nLower temp = lower msd(kind of)\n1.2kT, 1.3kT, 1.4kT clumped together but 1.3kT is on top\n0.9kT and 1.0kT indentical both taking turns but 0.9kT is on top\n\nAt 1.1den:\nLower temp = lower msd(with one exception)\n1.3kT higher msd than 1.4kT\n\nAt 1.2den:\nLower temp = lower msd(until after 1.1kT)\nThen order of least to greatest goes: 1.4kT, 1.2kT, 1.3kT(top)\n'